In [1]:
from sp import getSparcityPrior
from dsc import DeepSubspaceClustering
import scipy.io as sio

ourdata = sio.loadmat("/Users/xupeng.tong/Documents/Data/OriginalData/B_mean_2labels.mat")

inputX = ourdata['X']
# inputX = normalize(inputX, axis=0)
inputY = ourdata['Y'][0,:]
columnNames = ourdata['columnNames']

# getSparcityPrior(inputX, lambda1=0.01, lambda2=10)

In [2]:
import numpy as np

a = np.random.rand(5,5)
b = np.delete(a, (2), axis=1)

In [3]:
from sp_cvx import sparseCoefRecovery

C = sparseCoefRecovery(inputX.T, 'L1Perfect')

0


SolverError: Solver 'ECOS' failed. Try another solver.

In [ ]:
C

In [ ]:
# C = getSparcityPrior(inputX, lambda1=0.01, lambda2=0.1, epochs=100, print_step=1)